In [9]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
#from configs.RecursiveVPSDE.Markovian_fQuadSinHF.recursive_Markovian_PostMeanScore_fQuadSinHF2_LowFTh_T256_H05_tl_110data_StbleTgt  import get_config
from configs.RecursiveVPSDE.Markovian_fSinLog.recursive_Markovian_PostMeanScore_fSinLog_LowFTh_T256_H05_tl_110data_StbleTgt import get_config
#from configs.RecursiveVPSDE.Markovian_fBiPot.recursive_Markovian_PostMeanScore_fBiPot_LowFTh_T256_H05_tl_110data_StbleTgt import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [11]:
config = get_config()
root_dir ="/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/"
hermite_dir = root_dir + "ExperimentResults/Hermite/"
if "SinLog" in config.data_path:
    diradd = "SinLog"
elif "QuadSin" in config.data_path:
    diradd = "QuadSinHF"
elif "BiPot" in config.data_path:
    diradd = "BiPot"
if config.diffusion == 0.1:
        diradd+= "LowNoise"
elif config.diffusion == 10.:
    diradd += "HighNoise"
hermite_dir += diradd+"/"
print(hermite_dir)

/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/ExperimentResults/Hermite/SinLog/


In [12]:
for f in os.listdir(hermite_dir):
    assert "1024NPath" not in f
    assert "Diff" not in f
    if "MSE" in f:
        mse = pd.read_parquet(hermite_dir+f, engine="fastparquet")
    elif "est" in f:
        print(f)
        est =(np.load(hermite_dir+f, allow_pickle=True))
    elif "true" in f:
        true =(np.load(hermite_dir+f, allow_pickle=True))


In [13]:
print(mse, mse.dropna().index[np.argmin(mse.dropna().values.flatten())])
R = mse.dropna().index[np.argmin(mse.dropna().values.flatten())]

              0
index          
2      0.015540
3      0.036135
4      0.012242
5      0.027564
6      0.001686
7      0.001817
8      0.008302
9      0.001374
10     0.001147
11     0.001139
12     0.000863
13     0.000973
14     0.001401
15     0.001066
16     0.000604 16


In [14]:
from configs import project_config
num_paths = 10240
save_path = (
        project_config.ROOT_DIR + f"experiments/results/Hermite_f{diradd}_DriftEvalExp_{num_paths}NPaths_{config.deltaT:.3e}dT").replace(
    ".", "")
all_hermite_drifts = np.load(save_path + f"_{R}_drift_est.npy")
all_true_drifts=np.load(save_path + f"_{R}_true_drift.npy", allow_pickle=True)
all_true_paths = np.load(save_path + f"_{R}_true_paths.npy", allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/marcos/GitHubRepos/FractionalBrownianMotion/experiments/results/Hermite_fSinLog_DriftEvalExp_10240NPaths_3906e-03dT_16_drift_est.npy'

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(all_true_paths.flatten(), all_true_drifts.flatten(), label="True")
plt.scatter(all_true_paths.flatten(), all_hermite_drifts.flatten(), label="Hermite")
plt.legend()
plt.show()